ModuleNotFoundError: No module named 'haversine'


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from haversine import haversine

# Load preprocessed data
users_df = pd.read_csv("data/preprocessed_users.csv")
mechanics_df = pd.read_csv("data/preprocessed_mechanics.csv")
transactions_df = pd.read_csv("data/preprocessed_service_requests.csv")


users_df[['lat', 'long']] = users_df[['lat', 'long']].astype(float)
mechanics_df[['lat', 'long']] = mechanics_df[['lat', 'long']].astype(float)


df = transactions_df.merge(users_df, on="user_id").merge(mechanics_df, on="mechanic_id", suffixes=("_user", "_mech"))


df['distance_km'] = df.apply(lambda row: haversine(
    (row['lat_user'], row['long_user']),
    (row['lat_mech'], row['long_mech'])
), axis=1)


user_encoder = LabelEncoder()
mech_encoder = LabelEncoder()
service_encoder = LabelEncoder()

df['user_id_enc'] = user_encoder.fit_transform(df['user_id'])
df['mechanic_id_enc'] = mech_encoder.fit_transform(df['mechanic_id'])
df['service_type_enc'] = service_encoder.fit_transform(df['service_type'])


scaler = StandardScaler()
df[['distance_km', 'ratings', 'cost', 'experience_years', 'response_time']] = scaler.fit_transform(
    df[['distance_km', 'ratings', 'cost', 'experience_years', 'response_time']]
)

train, test = train_test_split(df, test_size=0.2, random_state=42)

print(" Data preprocessing completed successfully!")


✅ Data preprocessing completed successfully!


In [33]:

user_input = Input(shape=(1,), name="user_input")
mech_input = Input(shape=(1,), name="mechanic_input")
service_input = Input(shape=(1,), name="service_input")
distance_input = Input(shape=(1,), name="distance_input")
ratings_input = Input(shape=(1,), name="ratings_input")
experience_input = Input(shape=(1,), name="experience_input")
cost_input = Input(shape=(1,), name="cost_input")


embedding_dim = 16
user_embedding = Embedding(input_dim=len(user_encoder.classes_), output_dim=embedding_dim, name="user_embedding")(user_input)
mech_embedding = Embedding(input_dim=len(mech_encoder.classes_), output_dim=embedding_dim, name="mech_embedding")(mech_input)

user_vec = Flatten()(user_embedding)
mech_vec = Flatten()(mech_embedding)


concat_features = Concatenate()([user_vec, mech_vec, service_input, distance_input, ratings_input, experience_input, cost_input])


x = Dense(128, activation="relu")(concat_features)
x = Dropout(0.2)(x)
x = Dense(64, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(32, activation="relu")(x)


output = Dense(1, activation="sigmoid", name="output")(x)


model = Model(inputs=[user_input, mech_input, service_input, distance_input, ratings_input, experience_input, cost_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])





In [34]:

X_train = [
    train['user_id_enc'].values,
    train['mechanic_id_enc'].values,
    train['service_type_enc'].values,
    train['distance_km'].values,
    train['ratings'].values,
    train['experience_years'].values,
    train['cost'].values
]

y_train = (train['user_feedback'] >= 3).astype(int)  

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

print(" Model training completed successfully!")


Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 15s 35ms/step - accuracy: 0.9861 - loss: 0.1859 - val_accuracy: 1.0000 - val_loss: 0.0013
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 1.0000 - loss: 8.2853e-04 - val_accuracy: 1.0000 - val_loss: 2.2175e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 1.0000 - loss: 1.5501e-04 - val_accuracy: 1.0000 - val_loss: 8.6359e-05
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 1.0000 - loss: 9.7936e-05 - val_accuracy: 1.0000 - val_loss: 4.4858e-05
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 1.0000 - loss: 4.1574e-05 - val_accuracy: 1.0000 - val_loss: 2.7648e-05
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 1.0000 - loss: 3.0472e-05 - val_accuracy: 1.0000 - val_loss: 1.7305e-05
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 1.0000 - loss: 1.6248e-05 - val_accuracy: 1.0000 - val_loss: 1.2344e-05
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/

In [39]:
def recommend_mechanics(user_id, service_type):
  
    user = users_df[users_df['user_id'] == user_id]
    print(users_df['user_id'].dtype)  
    if user.empty:
        return "User not found!"

    user_lat, user_long = user.iloc[0]['lat'], user.iloc[0]['long']
    user_mechanics = eval(user.iloc[0]['mechanic_id'])  

    mechanics_df['distance_km'] = mechanics_df.apply(lambda row: haversine(
        (user_lat, user_long), (row['lat'], row['long'])
    ), axis=1)

 
    available_mechanics = mechanics_df[mechanics_df['service_types'].apply(lambda x: service_type in eval(x))]
    
    if available_mechanics.empty:
        return "No mechanics available for this service!"


    X_recommend = [
        np.array([user_encoder.transform([user_id])]*len(available_mechanics)),  # User IDs
        mech_encoder.transform(available_mechanics['mechanic_id']),  # Mechanic IDs
        service_encoder.transform([service_type]*len(available_mechanics)),  # Service Type
        available_mechanics['distance_km'].values,
        available_mechanics['ratings'].values,
        available_mechanics['experience_years'].values,
        available_mechanics['response_time'].values
    ]

    scores = model.predict(X_recommend).flatten()

  
    available_mechanics['recommendation_score'] = scores
    top_mechanics = available_mechanics.sort_values(by='recommendation_score', ascending=False).head(5)

  
    top_mechanics['user_rated'] = top_mechanics['mechanic_id'].apply(lambda x: x in user_mechanics)

    return top_mechanics[['mechanic_id', 'name', 'service_types', 'ratings', 'distance_km', 'recommendation_score', 'user_rated']]

recommended_mechanics = recommend_mechanics("user_0", "Oil Change")
print(recommended_mechanics)


object


KeyError: 'mechanic_id'

In [46]:
import pandas as pd
import numpy as np
from ast import literal_eval

def recommend_mechanics(user_id, service_type):
  
    user = users_df[users_df['user_id'] == user_id]
    
  
    if user.empty:
        return "User not found!"
    
   
    user_lat, user_long = user.iloc[0]['lat'], user.iloc[0]['long']

    preferred_mechanics = literal_eval(user.iloc[0]['preferred_mechanics'])  # Convert string to list

   
    past_services = literal_eval(user.iloc[0]['past_service_history'])  # Convert string to list of dicts
    past_mechanics = [service['mechanic_id'] for service in past_services]  # List of mechanic_ids user interacted with


    mechanics_df['distance_km'] = mechanics_df.apply(lambda row: haversine(
        (user_lat, user_long), (row['lat'], row['long'])
    ), axis=1)

   
    available_mechanics = mechanics_df[mechanics_df['service_types'].apply(lambda x: service_type in eval(x))]
    
    if available_mechanics.empty:
        return "No mechanics available for this service!"

    
    X_recommend = [
        np.array([user_encoder.transform([user_id])]*len(available_mechanics)),  # User IDs
        mech_encoder.transform(available_mechanics['mechanic_id']),  # Mechanic IDs
        service_encoder.transform([service_type]*len(available_mechanics)),  # Service Type
        available_mechanics['distance_km'].values,
        available_mechanics['ratings'].values,
        available_mechanics['experience_years'].values,
        available_mechanics['response_time'].values
    ]

 
    scores = model.predict(X_recommend).flatten()

    
    available_mechanics = available_mechanics.copy()
    
    available_mechanics.loc[:, 'recommendation_score'] = scores
    available_mechanics.loc[:, 'user_interacted'] = available_mechanics['mechanic_id'].apply(lambda x: x in past_mechanics)
    available_mechanics.loc[:, 'preferred_mechanic'] = available_mechanics['mechanic_id'].apply(lambda x: x in preferred_mechanics)

    top_mechanics = available_mechanics.sort_values(by='recommendation_score', ascending=False).head(5)

  
    top_mechanics['user_interacted'] = top_mechanics['mechanic_id'].apply(lambda x: x in past_mechanics)

   
    top_mechanics['preferred_mechanic'] = top_mechanics['mechanic_id'].apply(lambda x: x in preferred_mechanics)

   
    return top_mechanics[['mechanic_id', 'name', 'service_types', 'ratings', 'distance_km', 'recommendation_score', 'user_interacted', 'preferred_mechanic']]


recommended_mechanics = recommend_mechanics("user_0", "Oil Change")
print(recommended_mechanics)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step  
    mechanic_id                 name  \
1        mech_1  Christina Rodriguez   
328    mech_328         Ariel Dunlap   
311    mech_311          Lauren Cook   
315    mech_315      Michelle Duncan   
316    mech_316         Jeffrey Snow   

                                         service_types  ratings  distance_km  \
1    ['Oil Change', 'Tire Replacement', 'Transmissi...      4.7    26.937274   
328  ['Oil Change', 'General Service', 'Engine Repa...      3.7    49.787001   
311  ['Oil Change', 'Transmission Repair', 'Battery...      3.1    70.836107   
315  ['Suspension Fix', 'Brake Repair', 'Transmissi...      3.2    37.824340   
316  ['AC Service', 'Transmission Repair', 'Wheel A...      4.1    24.939018   

     recommendation_score  user_interacted  preferred_mechanic  
1                     1.0            False               False  
328                   1.0            False               False  
311                   1.0            False  

In [47]:
import joblib

# Save the trained model
joblib.dump(model, 'ml_model/mechanic_recommendation_model.pkl')


['ml_model/mechanic_recommendation_model.pkl']

In [48]:
from flask import Flask, request, jsonify
import joblib
import pandas as pd

# Initialize Flask app
app = Flask(__name__)

# Load the trained model
model = joblib.load('ml_model/mechanic_recommendation_model.pkl')

# Sample mechanics dataframe for demo (replace with your actual data)
mechanics_df = pd.DataFrame({
    'mechanic_id': ['mech_1', 'mech_328', 'mech_311', 'mech_315', 'mech_316'],
    'name': ['Christina Rodriguez', 'Ariel Dunlap', 'Lauren Cook', 'Michelle Duncan', 'Jeffrey Snow'],
    'service_types': [['Oil Change', 'Tire Replacement'], ['Oil Change', 'Engine Repair'], ['Suspension Fix'], ['Brake Repair'], ['AC Service']],
    'ratings': [4.7, 3.7, 3.1, 3.2, 4.1],
    'distance_km': [26.9, 49.8, 70.8, 37.8, 24.9]
})

# Define prediction function (you can use your actual logic here)
def recommend_mechanics(user_data, service_type):
    # Simulate a recommendation logic
    recommended_mechanics = mechanics_df[mechanics_df['service_types'].apply(lambda x: service_type in x)]
    recommended_mechanics['recommendation_score'] = recommended_mechanics['ratings'] / (recommended_mechanics['distance_km'] + 1)
    return recommended_mechanics[['mechanic_id', 'name', 'recommendation_score']]

# Define the API endpoint for prediction
@app.route('/recommend_mechanics', methods=['POST'])
def get_recommendations():
    try:
        data = request.get_json()
        user_data = data['user_data']  # Example user data
        service_type = data['service_type']
        
        # Get predictions based on user data and service type
        recommendations = recommend_mechanics(user_data, service_type)
        
        return jsonify(recommendations.to_dict(orient='records'))
    
    except Exception as e:
        return jsonify({'error': str(e)}), 400

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\anjal\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
